In [1]:
import pandas as pd
import numpy as np 
import warnings 
warnings.filterwarnings("ignore")

In [2]:
#1916-2016
movie_metadata=pd.read_csv("movie_metadata.csv")
data_movie=movie_metadata.copy()

data_movie=data_movie[["movie_title","genres","director_name","actor_1_name",
                       "actor_2_name","actor_3_name"]]

data_movie["genres"]=data_movie["genres"].str.replace("|"," ")


data_movie["movie_title"]=data_movie["movie_title"].str.lower()
data_movie["movie_title"]=data_movie["movie_title"].str.replace("part","")
data_movie["movie_title"]=data_movie["movie_title"].str.replace(":"," ")
data_movie["movie_title"]=data_movie["movie_title"].str.replace("-"," ")
data_movie["movie_title"]=data_movie["movie_title"].str.replace("  "," ")
data_movie["movie_title"]=data_movie["movie_title"].str.replace("   "," ")

data_movie["movie_title"]=data_movie["movie_title"].str.upper()
import roman 
movie_names=data_movie["movie_title"]
new_movies = []
for movie in movie_names:
    word_list = movie.split()
    new_word_list = []
    for word in word_list:
        try:
            number = roman.fromRoman(word)
            new_word_list.append(str(number))
        except roman.InvalidRomanNumeralError:
            new_word_list.append(word)
    new_movie = " ".join(new_word_list)
    new_movies.append(new_movie)

data_movie["movie_title"]=pd.DataFrame(new_movies)
data_movie["movie_title"]=data_movie["movie_title"].str.lower()
movie_1916_2016=data_movie.copy()

In [3]:
movie_1916_2016.sample(5)

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
3341,the haunting in connecticut 2 ghosts of georgia,Drama Horror Mystery Thriller,Tom Elkins,Abigail Spencer,Cicely Tyson,Brad James
3313,barbecue,Comedy,Eric Lavaine,Lambert Wilson,Julie Engelbrecht,Lionel Abelanski
3352,quo vadis,Biography Drama History Romance,Mervyn LeRoy,Peter Ustinov,Deborah Kerr,Robert Taylor
2440,blow out,Mystery Thriller,Brian De Palma,Nancy Allen,Dennis Franz,John McMartin
1866,the passion of the christ,Drama,Mel Gibson,Christo Jivkov,Maia Morgenstern,Hristo Shopov


In [4]:
#2017-2022
def get_movies(year):
    link="https://en.wikipedia.org/wiki/List_of_American_films_of_"
    link=link+str(year)
    import pandas as pd
    df1 = pd.read_html(link, header=0)[2]
    df2 = pd.read_html(link, header=0)[3]
    df3 = pd.read_html(link, header=0)[4]
    df4 = pd.read_html(link, header=0)[5]
    df=df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
    return df

movie_2017=get_movies(2017)
movie_2018=get_movies(2018)
movie_2019=get_movies(2019)
movie_2020=get_movies(2020)
movie_2021=get_movies(2021)
movie_2022=get_movies(2022)

def get_columns(data):
    data=data[["Title","Cast and crew"]]
    return data

movie_2017=get_columns(movie_2017)
movie_2018=get_columns(movie_2018)
movie_2019=get_columns(movie_2019)
movie_2020=get_columns(movie_2020)
movie_2021=get_columns(movie_2021)
movie_2022=get_columns(movie_2022)

In [5]:
#2017 - 2019 
def get_genre_1(x):
    from tmdbv3api import TMDb,Movie
    import requests

    tmdb_movie = Movie()
    tmdb = TMDb()
    
    tmdb.api_key = '35f4deb82d67cb00ced5e9dd1893a52d'
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

#2020-2022:

def get_genre_2(x):
    from tmdbv3api import TMDb,Movie
    import requests
    
    tmdb = TMDb() 
    tmdb_movie = Movie() 
    tmdb.api_key = '35f4deb82d67cb00ced5e9dd1893a52d'
    genres = []
    tmdb.api_key = '35f4deb82d67cb00ced5e9dd1893a52d'
    result = tmdb_movie.search(x)
    if not result:
          return np.NaN
    else:
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
            genre_str = " " 
            for i in range(0,len(data_json['genres'])):
                  genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            return np.NaN

            
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]


def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])


def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])
 
  
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [6]:
#2017
movie_2017['genres'] = movie_2017['Title'].map(lambda x: get_genre_1(str(x)))
movie_2017["director_name"]= movie_2017['Cast and crew'].map(lambda x: get_director(x))
movie_2017["actor_name1"]= movie_2017['Cast and crew'].map(lambda x: get_actor1(x))
movie_2017["actor_name2"]= movie_2017['Cast and crew'].map(lambda x: get_actor2(x)) 
movie_2017["actor_name3"]= movie_2017['Cast and crew'].map(lambda x: get_actor3(x))

In [7]:
movie_2017.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
148,Good Time,Ben Safdie & Joshua Safdie (directors); Joshua...,Thriller Crime,Ben Safdie & Joshua Safdie,Robert Pattinson,Jennifer Jason Leigh,Ben Safdie
167,American Assassin,Michael Cuesta (director); Stephen Schiff (scr...,Action Thriller,Michael Cuesta,Dylan O'Brien,Michael Keaton,Sanaa Lathan
212,LBJ,Rob Reiner (director); Joey Hartstone (screenp...,Drama History,Rob Reiner,Woody Harrelson,Richard Jenkins,Bill Pullman
229,Just Getting Started,Ron Shelton (director/screenplay); Morgan Free...,Action Comedy,Ron Shelton,Morgan Freeman,Tommy Lee Jones,Rene Russo
198,Wonderstruck,Todd Haynes (director); Brian Selznick (screen...,Drama,Todd Haynes,Oakes Fegley,Julianne Moore,Millicent Simmonds


In [8]:
#2018
movie_2018['genres'] = movie_2018['Title'].map(lambda x: get_genre_1(str(x)))
movie_2018["director_name"]= movie_2018['Cast and crew'].map(lambda x: get_director(x))
movie_2018["actor_name1"]= movie_2018['Cast and crew'].map(lambda x: get_actor1(x))
movie_2018["actor_name2"]= movie_2018['Cast and crew'].map(lambda x: get_actor2(x)) 
movie_2018["actor_name3"]= movie_2018['Cast and crew'].map(lambda x: get_actor3(x))


In [9]:
movie_2018.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
259,Ben Is Back,Peter Hedges (director/screenplay); Lucas Hedg...,Drama,Peter Hedges,Lucas Hedges,Julia Roberts,Kathryn Newton
203,My Hero Academia: Two Heroes,Kenji Nagasaki (director); Yōsuke Kuroda (scre...,Animation Action Adventure Fantasy,Kenji Nagasaki,Justin Briner,Christopher R. Sabat,Clifford Chapin
225,What They Had,Elizabeth Chomko (director/screenplay); Hilary...,Drama,Elizabeth Chomko,Hilary Swank,Michael Shannon,Robert Forster
227,Johnny English Strikes Again,David Kerr (director); William Davies (screenp...,Action Adventure Comedy,David Kerr,Rowan Atkinson,Olga Kurylenko,Ben Miller
226,Hunter Killer,"Donovan Marsh (director); Arne Schmidt, Jamie ...",Action Thriller,Donovan Marsh,Gerard Butler,Gary Oldman,Common


In [10]:
#2019
movie_2019['genres'] = movie_2019['Title'].map(lambda x: get_genre_1(str(x)))
movie_2019["director_name"]= movie_2019['Cast and crew'].map(lambda x: get_director(x))
movie_2019["actor_name1"]= movie_2019['Cast and crew'].map(lambda x: get_actor1(x))
movie_2019["actor_name2"]= movie_2019['Cast and crew'].map(lambda x: get_actor2(x)) 
movie_2019["actor_name3"]= movie_2019['Cast and crew'].map(lambda x: get_actor3(x))

In [11]:
movie_2019.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
103,Men in Black: International,F. Gary Gray (director); Art Marcum and Matt H...,Comedy Science Fiction Action,F. Gary Gray,Chris Hemsworth,Tessa Thompson,Kumail Nanjiani
180,Little Monsters,Abe Forsythe (director/screenplay); Lupita Nyo...,Adventure Fantasy Comedy Family,Abe Forsythe,Lupita Nyong'o,Alexander England,Josh Gad
68,Someone Great,Jennifer Kaytin Robinson (director/screenplay)...,Comedy Romance,Jennifer Kaytin Robinson,Gina Rodriguez,Brittany Snow,DeWanda Wise
138,The Angry Birds Movie 2,"Thurop Van Orman (director), John Rice (co-dir...",Animation Comedy Adventure Family,Thurop Van Orman,Jason Sudeikis,Josh Gad,Leslie Jones
87,The Professor,Wayne Roberts (director/screenplay); Johnny De...,Comedy Drama,Wayne Roberts,Johnny Depp,Rosemarie DeWitt,Danny Huston


In [12]:
#2020
movie_2020['genres'] = movie_2020['Title'].map(lambda x: get_genre_2(str(x)))
movie_2020["director_name"]= movie_2020['Cast and crew'].map(lambda x: get_director(str(x)))
movie_2020["actor_name1"]= movie_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))
movie_2020["actor_name2"]= movie_2020['Cast and crew'].map(lambda x: get_actor2(str(x))) 
movie_2020["actor_name3"]= movie_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [13]:
movie_2020.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
28,The Call of the Wild,Chris Sanders (director); Michael Green (scree...,Adventure Family Drama,Chris Sanders,Harrison Ford,Omar Sy,Cara Gee
80,Body Cam,Malik Vitthal (director); Richmond Riedel (scr...,Horror Thriller Mystery,Malik Vitthal,Mary J. Blige,Nat Wolff,David Zayas
126,The SpongeBob Movie: Sponge on the Run (Canadi...,"Tim Hill (director/screenplay); Tom Kenny, Awk...",NaN,Tim Hill,Tom Kenny,Awkwafina,Matt Berry
44,First Cow,Kelly Reichardt (director/screenplay); Jonatha...,Drama Western,Kelly Reichardt,John Magaro,Orion Hall,René Auberjonois
142,Phineas and Ferb the Movie: Candace Against th...,"Bob Bowen (director); Dan Povenmire, Jeff ""Swa...",Family Animation Science Fiction Comedy TV Movie,Bob Bowen,Vincent Martella,Ashley Tisdale,David Errigo Jr.


In [14]:
#2021
movie_2021['genres'] = movie_2021['Title'].map(lambda x: get_genre_2(str(x)))
movie_2021["director_name"]= movie_2021['Cast and crew'].map(lambda x: get_director(str(x)))
movie_2021["actor_name1"]= movie_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))
movie_2021["actor_name2"]= movie_2021['Cast and crew'].map(lambda x: get_actor2(str(x))) 
movie_2021["actor_name3"]= movie_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [15]:
movie_2021.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
359,Memoria,Apichatpong Weerasethakul (director/acreenplay...,Drama,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar
152,Zola,Janicza Bravo (director/screenplay); Jeremy O....,Comedy Crime,Janicza Bravo,Taylour Paige,Riley Keough,Nicholas Braun
23,PVT Chat,"Ben Hozie (director/screenplay); Julia Fox, Ni...",Drama,Ben Hozie,Julia Fox,Nikki Belfiglio,Austin Brown
332,Single All the Way,Michael Mayer (director); Chad Hodge (screenpl...,Romance Comedy,Michael Mayer,Michael Urie,Philemon Chambers,Luke Macfarlane
153,No Sudden Move,Steven Soderbergh (director); Ed Solomon (scre...,Crime Drama Mystery,Steven Soderbergh,Don Cheadle,Benicio del Toro,David Harbour


In [16]:
#2022
movie_2022['genres'] = movie_2022['Title'].map(lambda x: get_genre_2(str(x)))
movie_2022["director_name"]= movie_2022['Cast and crew'].map(lambda x: get_director(str(x)))
movie_2022["actor_name1"]= movie_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))
movie_2022["actor_name2"]= movie_2022['Cast and crew'].map(lambda x: get_actor2(str(x))) 
movie_2022["actor_name3"]= movie_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [17]:
movie_2022.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
180,Easter Sunday,"Jay Chandrasekhar (director); Ken Cheng, Kate ...",Comedy,Jay Chandrasekhar,Jo Koy,Jimmy O. Yang,Tia Carrere
3,Doctor Strange in the Multiverse of Madness,NaN,Fantasy Action Adventure,nan,nan,NaN,NaN
229,The Munsters,Rob Zombie (director/screenplay); Jeff Daniel ...,Comedy Fantasy,Rob Zombie,Jeff Daniel Phillips,Sheri Moon Zombie,Daniel Roebuck
49,No Exit,"Damien Power (director); Andrew Barrer, Gabrie...",Thriller Mystery,Damien Power,Havana Rose Liu,Danny Ramirez,Dennis Haysbert
144,Rise,Akin Omotoso (director); Arash Amel (screenpla...,Drama,Akin Omotoso,Uche Agada,Ral Agada,Jaden Osimuwa


In [18]:
#delete colums 
movie_2017=movie_2017.drop("Cast and crew",axis=1)
movie_2018=movie_2018.drop("Cast and crew",axis=1)
movie_2019=movie_2019.drop("Cast and crew",axis=1)
movie_2020=movie_2020.drop("Cast and crew",axis=1)
movie_2021=movie_2021.drop("Cast and crew",axis=1)
movie_2022=movie_2022.drop("Cast and crew",axis=1)

In [19]:
#new columns :
movie_1916_2016.rename(columns={"movie_title":"Title",
                               "genres":"genres",
                               "director_name":"director_name",
                               "actor_1_name":"actor_name1",
                               "actor_2_name":"actor_name2",
                               "actor_3_name":"actor_name3"},inplace=True)


movie_1916_2016["genres_director_actor"]=movie_1916_2016['actor_name1'] + ' ' + movie_1916_2016['actor_name2'] + ' '+ movie_1916_2016['actor_name3'] + ' '+ movie_1916_2016['director_name'] +' ' + movie_1916_2016['genres']
movie_2017["genres_director_actor"]=movie_2017['actor_name1'] + ' ' + movie_2017['actor_name2'] + ' '+ movie_2017['actor_name3'] + ' '+ movie_2017['director_name'] +' ' + movie_2017['genres']
movie_2018["genres_director_actor"]=movie_2018['actor_name1'] + ' ' + movie_2018['actor_name2'] + ' '+ movie_2018['actor_name3'] + ' '+ movie_2018['director_name'] +' ' + movie_2018['genres']
movie_2019["genres_director_actor"]=movie_2019['actor_name1'] + ' ' + movie_2019['actor_name2'] + ' '+ movie_2019['actor_name3'] + ' '+ movie_2019['director_name'] +' ' + movie_2019['genres']
movie_2020["genres_director_actor"]=movie_2020['actor_name1'] + ' ' + movie_2020['actor_name2'] + ' '+ movie_2020['actor_name3'] + ' '+ movie_2020['director_name'] +' ' + movie_2020['genres']
movie_2021["genres_director_actor"]=movie_2021['actor_name1'] + ' ' + movie_2021['actor_name2'] + ' '+ movie_2021['actor_name3'] + ' '+ movie_2021['director_name'] +' ' + movie_2021['genres']
movie_2022["genres_director_actor"]=movie_2022['actor_name1'] + ' ' + movie_2022['actor_name2'] + ' '+ movie_2022['actor_name3'] + ' '+ movie_2022['director_name'] +' ' + movie_2022['genres']

In [20]:
#concat
all_movies = pd.concat([movie_1916_2016, movie_2017, movie_2018, movie_2019, movie_2020, movie_2021, movie_2022])

In [21]:
all_movies.sample(5)

,Title,genres,director_name,actor_name1,actor_name2,actor_name3,genres_director_actor
323,Friend of the World,Comedy Science Fiction Horror,Brian Patrick Butler,Nick Young,Alexandra Slade,Michael C. Burgess,Nick Young Alexandra Slade Michael C. Burgess ...
172,She's in Portland,Drama,Marc Carlini,Tommy Dewey,François Arnaud,Minka Kelly,Tommy Dewey François Arnaud Minka Kelly Marc C...
2027,the fisher king,Comedy Drama Fantasy,Terry Gilliam,Jeff Bridges,David Hyde Pierce,Harry Shearer,Jeff Bridges David Hyde Pierce Harry Shearer T...
3628,novocaine,Comedy Crime Drama Thriller,David Atkins,Chelcie Ross,Lynne Thigpen,Polly Noonan,Chelcie Ross Lynne Thigpen Polly Noonan David ...
5,The Sonata,Horror Thriller Mystery,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer,Freya Tingley Simon Abkarian Rutger Hauer Andr...


In [22]:
all_movies.shape

(6686, 7)

In [23]:
all_movies=all_movies.dropna()

In [24]:
all_movies.shape

(6417, 7)

In [25]:
all_movies.drop_duplicates( inplace=True)

In [26]:
all_movies.shape

(6295, 7)

In [27]:
#saving data :
movie_1916_2016.to_csv("movie_1916_2016.csv")
movie_2017.to_csv("movie_2017.csv")
movie_2018.to_csv("movie_2018.csv")
movie_2019.to_csv("movie_2019.csv")
movie_2020.to_csv("movie_2020.csv")
movie_2021.to_csv("movie_2021.csv")
movie_2022.to_csv("movie_2022.csv")



all_movies.to_csv("all_movies.csv")